In [1]:
import locationtagger
import nltk
from pymongo import MongoClient
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import os
# from collections import Counter
from dotenv import load_dotenv
load_dotenv()


# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package

True

In [2]:

def getTweetData(tweet):
    try:
        tweetTime = tweet.find_element(
            By.XPATH, ".//time").get_attribute('datetime')
    except NoSuchElementException:
        return
    try:
        # This only gets one image, what if there are more?
        tweetImage = tweet.find_element(
            By.XPATH, ".//div[1]/div[1]//div[2]/div[2]//img").get_attribute("src")
    except NoSuchElementException:
        # print("no image")
        tweetImage = "No Image"
    tweetText = tweet.find_element(
        By.XPATH, ".//div[1]/div[1]/div[2]/div[2]/div[2]").text

    tweetInfo = [tweetText, tweetTime, tweetImage]
    return tweetInfo


def ScraperMain():
    twitterUsername = os.getenv('TW_USERNAME')
    twitterPassword = os.getenv('TW_PASSWORD')

    # Setting up driver
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get('https://www.twitter.com/login')
    sleep(3)
    # Finding and inputing username
    username = driver.find_element(
        By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[5]/label[1]/div[1]/div[2]/div[1]/input[1]")
    username.send_keys(twitterUsername)
    username.send_keys(Keys.RETURN)
    sleep(1)
    # Finding and inputing Password
    # mypassword = getpass()
    password = driver.find_element(
        By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/label[1]/div[1]/div[2]/div[1]/input[1]")
    password.send_keys(twitterPassword)
    password.send_keys(Keys.RETURN)
    sleep(5)

    # Selecting and searching required police section
    search = driver.find_element(
        By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[2]/main[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/form[1]/div[1]/div[1]/div[1]/div[1]/label[1]/div[2]/div[1]/input[1]")
    search.send_keys('@TPSOperations')
    search.send_keys(Keys.RETURN)
    sleep(2)

    # Click required page
    SearchQuery = driver.find_element(
        By.XPATH, "//span[contains(text(),'Toronto Police Operations')]")
    SearchQuery.click()

    # Storing Tweet data in list
    tweetData = []
    tweetIds = set()
    lastPos = driver.execute_script("return window.pageYOffset;")
    scrolling = True
    maxDate = False

    while (scrolling and not (maxDate)):
        # Find Tweets
        TPOPageTweets = driver.find_elements(
            By.XPATH, '//article[@data-testid="tweet"]')
        sleep(1)
        for tweet in TPOPageTweets[-100:]:
            currentTweetInfo = getTweetData(tweet)
            if currentTweetInfo:
                tweetId = "".join(currentTweetInfo)
                if tweet not in tweetIds and currentTweetInfo[0] != '' and ":" in currentTweetInfo[0] and 'Good night' not in currentTweetInfo[0] and 'Good afternoon' not in currentTweetInfo[0] and 'Good morning' not in currentTweetInfo[0] and 'Good Night' not in currentTweetInfo[0]:
                    tweetIds.add(tweet)
                    currentTweetInfo.append(tweetId)
                    tweetData.append(currentTweetInfo)
                    if "2023-05-20" in currentTweetInfo[1]:
                        maxDate = True
                        break

        scrollAttempt = 0
        while True:
            driver.execute_script('window.scrollBy(0,3000);')
            sleep(2)
            currPos = driver.execute_script("return window.pageYOffset;")
            if maxDate:
                break
            if lastPos == currPos:
                scrollAttempt += 1
                if scrollAttempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(2)
            else:
                lastPos = currPos
                break
    print("Tweets Scraped")
    return tweetData

In [5]:
# Storing Tweet data in list
tweetData = []
tweetIds = set()
lastPos = driver.execute_script("return window.pageYOffset;")
scrolling = True
maxDate = False

while (scrolling and not (maxDate)):
    # Find Tweets
    TPOPageTweets = driver.find_elements(
        By.XPATH, '//article[@data-testid="tweet"]')
    sleep(1)
    for tweet in TPOPageTweets[-100:]:
        currentTweetInfo = getTweetData(tweet)
        if currentTweetInfo:
            tweetId = "".join(currentTweetInfo)
            if tweet not in tweetIds:
                tweetIds.add(tweet)
                tweetData.append([currentTweetInfo, tweetId])
                # print(currentTweetInfo[1])
                if "2023-04-01" in currentTweetInfo[1]:
                    maxDate = True
                    break

    scrollAttempt = 0
    while True:
        driver.execute_script('window.scrollBy(0,3000);')
        sleep(2)
        currPos = driver.execute_script("return window.pageYOffset;")
        if maxDate:
            break
        if lastPos == currPos:
            scrollAttempt += 1
            if scrollAttempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            lastPos = currPos
            break

NameError: name 'driver' is not defined

In [4]:
def sortData(scrapedTweet):
    # List: catagory (Missing...), Status(Update, Located)
    computedData = []
    counter = 0
    for i in range(len(scrapedTweet)):
        dataObject = {}
        if (scrapedTweet[i][0].find('\n') != -1):
            # Getting Status:
            dataObject["Status"] = scrapedTweet[i][0].splitlines()[0].split(":")[0]
            # Getting Updates (?LOCATED)...
            try:
                dataObject["Updates"] = scrapedTweet[i][0].splitlines()[0].split(":")[1]
            except IndexError:
                dataObject["Updates"] = ""
            # currentLocation could be the currentIdentity
            currentLocation = scrapedTweet[i][0].splitlines()[1].split(", ")
            # Checking whether we have a Location or an identity
            if any(char.isdigit() for char in currentLocation):
                dataObject["Name"] = currentLocation[0]
                if currentLocation[1].isnumeric():
                    dataObject["Age"] = currentLocation[1]
                else:
                    dataObject["Age"] = ""
                description = scrapedTweet[i][0].splitlines()
                del description[0:2]
                #Extracting Location from Description
                lists = scrapedTweet[i][0].splitlines()
                del lists[0:2]
                try:
                    partOfDescription = lists[0] + " " + lists[1]
                except IndexError:
                    partOfDescription = scrapedTweet[i][0]                   
                dataObject["Location"] = locationtagger.find_locations(text = partOfDescription).other
            else:
                try:
                    dataObject["Location"] = scrapedTweet[i][0].splitlines()[1]
                except IndexError:
                    dataObject["Location"] = ""
                description = scrapedTweet[i][0].splitlines()
                del description[0:1]
            if dataObject["Location"] is not None:
                locationcheck = dataObject["Location"]
                if type(dataObject["Location"]) == list and bool(dataObject["Location"]):
                    res = max(locationcheck, key = len)
                    locationcheck = res
                if "and" in locationcheck:
                    locationcheck = locationcheck.replace("and", "")
                if "&" in locationcheck:
                    locationcheck = locationcheck.replace("&", "")
                geolocator = Nominatim(user_agent="Twitter_Scraper")
                geocode = lambda query: geolocator.geocode("%s, Toronto ON" % query)
                geocode2 = RateLimiter(geocode, min_delay_seconds=0.001)
                sleep(1)
                locationcor = geocode2(locationcheck)
                if locationcor is not None:
                    locationcor2 = [locationcor.latitude, locationcor.longitude]
                    dataObject["LocationGoeCode"] = locationcor2
                else:
                    dataObject["LocationGoeCode"] = []
                    
            else:
                dataObject["LocationGoeCode"] = ["No Location Provided"]
            dataObject["TweetedTime"] = scrapedTweet[i][1]
            dataObject["ImageUrl"] = scrapedTweet[i][2]
            dataObject["Description"] = description
            computedData.append(dataObject)
            counter = counter + 1
            print(counter)
            print(dataObject['Location'])
            print(dataObject['LocationGoeCode'])
        else:
            print("Tweet Skipped: ", scrapedTweet[i])
    print("Tweets Sorted")
    return(computedData)
        


In [5]:

def addToDB(computedData):
    # Get the database using the method we defined in pymongo_test_insert file
    # from pymongo_get_database import get_database
    MONGODB_PASS = os.getenv('MONGODB_PASS')

    # Provide the mongodb atlas url to connect python to mongodb using pymongo
    CONNECTION_STRING = "mongodb+srv://OCMB1:" + MONGODB_PASS + \
        "@cluster0.e5wloiz.mongodb.net/?retryWrites=true&w=majority"

    # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
    client = MongoClient(CONNECTION_STRING)

    # print(client.drop_database('Toronto_Police_Crime_Report'))
    # Create the database for our example (we will use the same database throughout the tutorial
    if 'Toronto_Police_Crime_Report' in client.list_database_names():
        client.drop_database('Toronto_Police_Crime_Report')

    db = client['Toronto_Police_Crime_Report']
    collection_name = db["tweetsData"]
    res = collection_name.insert_many(computedData)
    print("Added To DataBase")

In [6]:
tweets = ScraperMain()
# sortedData = sortData(tweets)
# addToDB(sortedData)

Tweets Scraped


In [9]:
len(tweets)

849

In [10]:
sortedData = sortData(tweets)


1
Carlaw Ave & Danforth Ave
[43.6783505, -79.3474623]
2
Jane St & Shoreham Dr
[43.7689691, -79.5205825]
3
Jane St & Shoreham Dr
[43.7689691, -79.5205825]
4
['July', 'the Shuter St & Victoria St', 'Shuter St', 'Victoria St']
[]
5
['July', 'the Shuter St & Victoria St', 'Shuter St', 'Victoria St']
[]
6
Kipling Ave & Dundas St W
[43.6401695, -79.535166]
7
Kipling Ave & Dundas St W
[43.6401695, -79.535166]
8
Jameson Ave & King St W
[43.6370522, -79.4361987]
9
Jameson Ave & King St W
[43.6370522, -79.4361987]
10
['June', 'Danforth Ave', 'Hillcrest Ave', 'Confederation', 'Greenwood Ave']
[43.768606, -79.4073178]
11
[]
[43.6534817, -79.3839347]
12
[]
[43.6534817, -79.3839347]
13
['July', 'the Spadina Rd & Dupont St', 'Spadina Rd', 'Dupont St']
[]
14
Jarvis St & Wellesley St
[43.6689516, -79.3794474]
15
Jarvis St & Wellesley St
[43.6689516, -79.3794474]
16
['July', 'the Keele St & Pond Rd', 'Keele St', 'Pond Rd']
[43.7716768, -79.4928525]
17
['July', 'Finch Ave', 'Bridletowne']
[43.80064679999

RateLimiter caught an error, retrying (0/2 tries). Called with (*('6:20 p.m.',), **{}).
Traceback (most recent call last):
  File "c:\Users\LENOVO\OneDrive\Documents\My_projects\OCM\Twitter_Scraper\venv\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\LENOVO\OneDrive\Documents\My_projects\OCM\Twitter_Scraper\venv\Lib\site-packages\urllib3\connection.py", line 454, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 1375, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
 

90
6:20 p.m.
[]


RateLimiter caught an error, retrying (0/2 tries). Called with (*('6:20 p.m.',), **{}).
Traceback (most recent call last):
  File "c:\Users\LENOVO\OneDrive\Documents\My_projects\OCM\Twitter_Scraper\venv\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\LENOVO\OneDrive\Documents\My_projects\OCM\Twitter_Scraper\venv\Lib\site-packages\urllib3\connection.py", line 454, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 1375, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
 

91
6:20 p.m.
[]
92
ROAD CLOSURES
[]
93
- all roads reopened in area
[]
94
- DSO Inspector Abdel-Malik to provide an update on Shooting that occurred in the Morningside Ave. and Danzig St. area 
[]
95
3pm
[]
96
3pm
[]
97
['July', 'the York Mills Rd & Victoria Park Ave', 'York Mills Rd', 'Victoria Park Ave']
[]
98
[]
[43.6534817, -79.3839347]
99
[]
[43.6534817, -79.3839347]
100
[]
[43.6534817, -79.3839347]
101
[]
[43.6534817, -79.3839347]
102
River St & Dundas St E
[43.6610909, -79.3582849]
103
River St & Dundas St E
[43.6610909, -79.3582849]


...

322
Wilson Ave & Ridge Rd
[]


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Wilson Ave  Ridge Rd',), **{}).
Traceback (most recent call last):
  File "c:\Users\LENOVO\OneDrive\Documents\My_projects\OCM\Twitter_Scraper\venv\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\LENOVO\OneDrive\Documents\My_projects\OCM\Twitter_Scraper\venv\Lib\site-packages\urllib3\connection.py", line 454, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 1375, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^

323
Wilson Ave & Ridge Rd
[]
324
McCowan Rd & Lawrence Ave E
[43.7553207, -79.2459358]
325
- several scenes located on Ellesmere Rd between McCowan Rd & Markham Rd and
[]
326
- suspect in custody
[]
327
- several scenes located on Ellesmere Rd between McCowan Rd & Markham Rd and
[]
328
- suspect in custody
[]
329
['June', 'the Dibgate Blvd & Chartland Blvd', 'Dibgate Blvd', 'Chartland Blvd']
[]
330
Kennedy Rd & Sheppard Ave E
[43.783021, -79.2887]
331
Kennedy Rd & Sheppard Ave E
[43.783021, -79.2887]
332
['Thursday, June', 'the Yonge Street Gerrard Street East']
[43.6623282, -79.3659027]
333
[]
[43.6534817, -79.3839347]
334
['June', 'the Keele St & Tilbury Dr', 'Keele St', 'Tilbury Dr']
[]
335
[]
[43.6534817, -79.3839347]
336
SHOOTING
[]
337
@CityofToronto
[]
338
Bellamy Rd N & Lawrence Ave E
[43.7577661, -79.2357505]
339
Bellamy Rd N & Lawrence Ave E
[43.7577661, -79.2357505]
340
Finch Ave E between Kennedy Rd & Midland Ave
[]
341
['June', 'the Wynford Drive Eglinton Avenue', 'Wynford

RateLimiter caught an error, retrying (0/2 tries). Called with (*('10:45 p.m.',), **{}).
Traceback (most recent call last):
  File "c:\Users\LENOVO\OneDrive\Documents\My_projects\OCM\Twitter_Scraper\venv\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\LENOVO\OneDrive\Documents\My_projects\OCM\Twitter_Scraper\venv\Lib\site-packages\urllib3\connection.py", line 454, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 1375, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1008.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^


421
10:45 p.m.
[]
422

[43.6534817, -79.3839347]
423
Sherbourne St & Gerrard St
[]
424
Sherbourne St & Gerrard St
[]
425
Sherbourne St & Gerrard St
[]
426
Keele St & Downview Park Blvd
[]
427
Keele St & Downview Park Blvd
[]
428
Royal York & Dixon Rd
[43.6998684, -79.5335993]
429
Royal York & Dixon Rd
[43.6998684, -79.5335993]
430
['June', 'Yorkville Ave &', 'Yonge St', 'Yorkville Ave']
[43.6707824, -79.3929111]
431
['Nevaeh']
[]
432
Allen Gardens
[]
433
Allen Gardens
[]
434
['approximately two weeks ago', 'the Keele St & Finch', 'Keele St', 'Finch Ave']
[]
435
[]
[43.6534817, -79.3839347]
436
['June', 'Islington Ave', 'HWY']
[43.6344249, -79.5191824]
437
Dixon Rd & St Phillips Rd
[43.7000368, -79.5344647]
438
['Tuesday, June', 'WRPSToday']
[]
439
[]
[43.6534817, -79.3839347]
440
['Friday, June', 'the Kennedy Rd McNicoll Av', 'McNicoll Av']
[]
441
Lake Shore Blvd + Lower Jarvis
[43.6454359, -79.3696674]
442
Lake Shore Blvd + Lower Jarvis
[43.6454359, -79.3696674]
443
Chalkfarm Dr + Jan

In [11]:
addToDB(sortedData)

Added To DataBase


In [8]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Twitter")
geocode = lambda query: geolocator.geocode("%s, Toronto ON" % query)
location = geocode("Victoria Park Van Horne")
location2 = [location.latitude, location.longitude]
location2
# print((location.latitude, location.longitude))

[43.7890415, -79.330173]

In [21]:
import requests

# Replace YOUR_API_KEY with your actual API key. Sign up and get an API key on https://www.geoapify.com/ 
API_KEY = "e904244da7514e13a457430791adcf16"

# Define the address to geocode
address = "Jane St Wilson Ave area, Toronto, CA"

# Build the API URL
url = f"https://api.geoapify.com/v1/geocode/search?text={address}&limit=1&apiKey={API_KEY}"

# Send the API request and get the response
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    # Parse the JSON data from the response
    data = response.json()

    # Extract the first result from the data
    result = data["features"][0]

    # Extract the latitude and longitude of the result
    latitude = result["geometry"]["coordinates"][1]
    longitude = result["geometry"]["coordinates"][0]

    print(f"Latitude: {latitude}, Longitude: {longitude}")
else:
    print(f"Request failed with status code {response.status_code}")

Latitude: 43.64877, Longitude: -79.38171
